# Predicting stock and stock price index movement using Random Forest Prediction Model

## Importing Libraries

In [2]:
import pandas as pd
import pandas_datareader.data as web
import datetime
import numpy as np
from talib.abstract import *
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
import plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
import cufflinks as cf
init_notebook_mode()

ImportError: No module named 'talib'

In [ ]:
plotly.tools.set_credentials_file(username='samarsrivastava44', api_key='••••••••••')

### Research Data used in this experiment consists of 3 stock price indices (S&P 500, Dow Jones & NASDAQ-100) and 3 stocks (Apple, Amazon & Google)

In [53]:
stock = ['AAPL','AMZN','GOOG']
index = ['GSPC','NDX','DJI']

## Loading of datasets

In [56]:
stocks = {}
indices = {}
for i in stock:
    stocks[i] = pd.read_csv('Datasets/Stocks/'+i+'.csv')
for i in index:
    indices[i]  = pd.read_csv('Datasets/Stock Indices/'+i+'.csv')

In [57]:
stocks['AAPL'].head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2007-01-03,12.327143,12.368571,11.700000,11.971429,10.609749,309579900
1,2007-01-04,12.007143,12.278571,11.974286,12.237143,10.845238,211815100
2,2007-01-05,12.252857,12.314285,12.057143,12.150000,10.768006,208685400
3,2007-01-08,12.280000,12.361428,12.182858,12.210000,10.821184,199276700
4,2007-01-09,12.350000,13.282857,12.164286,13.224286,11.720100,837324600


In [58]:
stocks['AMZN'].head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2007-01-03,38.680000,39.060001,38.049999,38.700001,38.700001,12405100
1,2007-01-04,38.590000,39.139999,38.259998,38.900002,38.900002,6318400
2,2007-01-05,38.720001,38.790001,37.599998,38.369999,38.369999,6619700
3,2007-01-08,38.220001,38.310001,37.169998,37.500000,37.500000,6783000
4,2007-01-09,37.599998,38.060001,37.340000,37.779999,37.779999,5703000


In [59]:
stocks['GOOG'].head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2007-01-03,231.494354,236.789917,229.065155,232.284210,232.284210,15513200
1,2007-01-04,232.984665,240.411362,232.661758,240.068588,240.068588,15877700
2,2007-01-05,239.691040,242.174881,237.510223,242.020889,242.020889,13833500
3,2007-01-08,242.269272,243.352234,239.542007,240.227554,240.227554,9570600
4,2007-01-09,241.156509,242.547470,239.045242,241.181351,241.181351,10832700


In [60]:
indices['NDX'].head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2007-01-03,1769.219971,1793.050049,1731.709961,1759.369995,1759.369995,2435280000
1,2007-01-04,1763.150024,1799.069946,1757.000000,1792.910034,1792.910034,2104210000
2,2007-01-05,1787.790039,1787.790039,1770.290039,1785.300049,1785.300049,2060360000
3,2007-01-08,1786.390015,1795.449951,1776.989990,1787.140015,1787.140015,1905620000
4,2007-01-09,1791.750000,1802.439941,1776.729980,1795.630005,1795.630005,2144160000


In [61]:
indices['GSPC'].head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2007-01-03,1418.030029,1429.420044,1407.859985,1416.599976,1416.599976,3429160000
1,2007-01-04,1416.599976,1421.839966,1408.430054,1418.339966,1418.339966,3004460000
2,2007-01-05,1418.339966,1418.339966,1405.750000,1409.709961,1409.709961,2919400000
3,2007-01-08,1409.260010,1414.979980,1403.969971,1412.839966,1412.839966,2763340000
4,2007-01-09,1412.839966,1415.609985,1405.420044,1412.109985,1412.109985,3038380000


In [62]:
indices['DJI'].head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2007-01-03,12459.540039,12580.349609,12404.820313,12474.519531,12474.519531,327200000
1,2007-01-04,12473.160156,12510.410156,12403.860352,12480.690430,12480.690430,259060000
2,2007-01-05,12480.049805,12480.129883,12365.410156,12398.009766,12398.009766,235220000
3,2007-01-08,12392.009766,12445.919922,12337.370117,12423.490234,12423.490234,223500000
4,2007-01-09,12424.769531,12466.429688,12369.169922,12416.599609,12416.599609,225190000


In [63]:
len(stocks['AMZN'])

2769

In [64]:
len(stocks['AAPL'])

2769

In [65]:
len(stocks['GOOG'])

2769

In [66]:
len(indices['GSPC'])

2769

In [67]:
len(indices['NDX'])

2769

In [68]:
len(indices['DJI'])

2769

## Data Refining

In [69]:
for i,j in enumerate(stocks):
    stocks[j].columns = [s.lower() for s in stocks[j].columns]
    stocks[j].volume = stocks[j].volume.apply(lambda x: float(x))
for i,j in enumerate(indices):
    indices[j].columns = [s.lower() for s in indices[j].columns]
    indices[j].volume = indices[j].volume.apply(lambda x: float(x))

## Calculation of Technical Indicators 

In [70]:
def get_stocks_indicators(stocks, period):
    stocks_indicators = {}
    for i in stocks:
        features = pd.DataFrame(SMA(stocks[i], timeperiod=10))
        features.columns = ['sma_10']
        features['mom_10'] = pd.DataFrame(MOM(stocks[i],10))
        features['wma_10'] = pd.DataFrame(WMA(stocks[i],10))
        features = pd.concat([features,STOCHF(stocks[i], 
                                          fastk_period=14, 
                                          fastd_period=3)],
                             axis=1)
        features['macd'] = pd.DataFrame(MACD(stocks[i], fastperiod=12, slowperiod=26)['macd'])
        features['rsi'] = pd.DataFrame(RSI(stocks[i], timeperiod=14))
        features['willr'] = pd.DataFrame(WILLR(stocks[i], timeperiod=14))
        features['cci'] = pd.DataFrame(CCI(stocks[i], timeperiod=14))
        features['adosc'] = pd.DataFrame(ADOSC(stocks[i], fastperiod=3, slowperiod=10))
        features['pct_change'] = ROC(stocks[i], timeperiod=period)
        features['pct_change'] = features['pct_change'].shift(-period)
        features['pct_change'] = features['pct_change'].apply(lambda x: '1' if x > 0 else '0' if x <= 0 else np.nan)
        features = features.dropna()
        #features = features.iloc[np.where(features.index=='1998-5-5')[0][0]:np.where(features.index=='2015-5-5')[0][0]]
        stocks_indicators[i] = features
    return stocks_indicators

def get_indices_indicators(indices, period):
    indices_indicators = {}
    for i in indices:
        features = pd.DataFrame(SMA(indices[i], timeperiod=10))
        features.columns = ['sma_10']
        features['mom_10'] = pd.DataFrame(MOM(indices[i],10))
        features['wma_10'] = pd.DataFrame(WMA(indices[i],10))
        features = pd.concat([features,STOCHF(indices[i], 
                                          fastk_period=14, 
                                          fastd_period=3)],
                             axis=1)
        features['macd'] = pd.DataFrame(MACD(indices[i], fastperiod=12, slowperiod=26)['macd'])
        features['rsi'] = pd.DataFrame(RSI(indices[i], timeperiod=14))
        features['willr'] = pd.DataFrame(WILLR(indices[i], timeperiod=14))
        features['cci'] = pd.DataFrame(CCI(indices[i], timeperiod=14))
        features['adosc'] = pd.DataFrame(ADOSC(indices[i], fastperiod=3, slowperiod=10))
        features['pct_change'] = ROC(indices[i], timeperiod=period)
        features['pct_change'] = features['pct_change'].shift(-period)
        features['pct_change'] = features['pct_change'].apply(lambda x: '1' if x > 0 else '0' if x <= 0 else np.nan)
        features = features.dropna()
        #features = features.iloc[np.where(features.index=='1998-5-5')[0][0]:np.where(features.index=='2015-5-5')[0][0]]
        indices_indicators[i] = features
    return indices_indicators

In [71]:
stocks_indicators = get_stocks_indicators(stocks, 1)
indices_indicators = get_indices_indicators(indices, 1)

In [72]:
stocks_indicators['AAPL'].head()

,sma_10,mom_10,wma_10,fastk,fastd,macd,rsi,willr,cci,adosc,pct_change
33,12.223143,0.721429,12.268441,95.625898,77.628692,-0.158363,61.901279,-4.374102,307.160552,1.170275e+08,1
34,12.271143,0.480000,12.370987,83.647820,87.520448,-0.104330,62.580599,-16.352180,258.654956,1.322122e+08,0
35,12.312429,0.412857,12.453377,78.113240,85.795653,-0.065821,60.920258,-21.886760,167.400412,9.737425e+07,0
36,12.387286,0.748571,12.513714,71.069213,77.610091,-0.041282,58.782651,-28.930787,105.206129,6.217692e+07,0
37,12.373714,-0.135714,12.441481,9.473674,52.885375,-0.073780,44.904824,-90.526326,-56.754097,-2.319024e+07,1


In [80]:
stocks_indicators['AMZN'].head()

,sma_10,mom_10,wma_10,fastk,fastd,macd,rsi,willr,cci,adosc,pct_change
33,39.826,2.989998,40.279636,90.513759,94.360388,0.822374,66.005718,-9.486241,140.013235,8.266044e+06,0
34,40.028,2.020000,40.493091,81.203008,89.057078,0.866861,63.556462,-18.796992,119.747659,6.687418e+06,0
35,40.196,1.680001,40.629818,76.673021,82.796596,0.874287,61.477678,-23.326979,82.599828,3.988678e+06,1
36,40.412,2.160000,40.754182,77.235783,78.370604,0.878119,62.084739,-22.764217,68.879754,2.778825e+06,0
37,40.410,-0.019996,40.466546,12.912119,55.606974,0.707582,46.060474,-87.087881,-49.107344,-6.248500e+05,1


In [74]:
stocks_indicators['GOOG'].head()

,sma_10,mom_10,wma_10,fastk,fastd,macd,rsi,willr,cci,adosc,pct_change
33,231.773538,2.175858,232.300837,40.870785,34.542740,-2.654167,52.757765,-59.129215,51.637777,3.329435e+06,0
34,232.063652,2.901138,233.139744,69.479639,47.949063,-2.308614,52.750632,-30.520361,124.882299,1.920714e+06,0
35,232.043285,-0.203674,233.453522,53.388125,54.579516,-2.218828,49.023207,-46.611875,55.031348,2.158679e+05,0
36,232.194302,1.510177,233.257073,33.915172,52.260979,-2.348682,45.274905,-66.084828,-0.515463,-2.522093e+06,0
37,231.721379,-4.729233,231.573568,4.316158,30.539818,-3.064048,36.693862,-95.683842,-186.460216,-7.866435e+06,1


In [75]:
indices_indicators['GSPC'].head()

,sma_10,mom_10,wma_10,fastk,fastd,macd,rsi,willr,cci,adosc,pct_change
33,1449.898035,9.630005,1452.778398,90.030886,93.716410,9.479183,63.938589,-9.969114,105.134662,3.813132e+09,0
34,1450.534033,6.359985,1453.956938,82.774842,90.142725,9.351993,62.698674,-17.225158,86.347828,3.617812e+09,0
35,1450.822022,2.879882,1454.076194,65.549276,79.451668,8.731744,57.695789,-34.450724,30.682218,2.979242e+09,0
36,1451.953015,11.309936,1453.812189,59.508965,69.277694,8.001107,56.008133,-40.491035,11.735103,2.158290e+09,0
37,1448.520020,-34.329956,1444.191648,13.333345,46.130528,3.322564,29.932484,-86.666655,-271.080636,7.487103e+08,1


In [76]:
indices_indicators['DJI'].head()

,sma_10,mom_10,wma_10,fastk,fastd,macd,rsi,willr,cci,adosc,pct_change
33,12689.221973,72.100586,12719.631250,77.853229,90.357251,68.713418,61.332950,-22.146771,86.549491,2.792817e+08,0
34,12691.136914,19.149414,12719.048988,57.681244,77.446350,63.239760,55.514281,-42.318756,27.456111,2.231410e+08,0
35,12692.121973,9.850586,12711.111452,42.842532,59.459002,55.156259,51.633712,-57.157468,-31.642342,1.606483e+08,0
36,12697.264942,51.429688,12700.227415,36.982098,45.835292,46.980283,50.142915,-63.017902,-38.287711,8.651410e+07,0
37,12663.633984,-336.309571,12612.768377,18.338660,32.721097,6.852474,27.106288,-81.661340,-293.681858,-1.685131e+07,1


In [77]:
indices_indicators['NDX'].head()

,sma_10,mom_10,wma_10,fastk,fastd,macd,rsi,willr,cci,adosc,pct_change
33,1810.197998,46.069946,1817.122905,100.000000,97.787830,6.109130,65.730631,-0.000000,147.818665,2.972300e+09,1
34,1813.736999,35.390015,1823.694172,93.356634,97.030925,8.482505,67.399671,-6.643366,157.399472,3.264304e+09,0
35,1816.659998,29.229981,1828.427448,84.848543,92.735059,9.721222,64.325302,-15.151457,114.328567,3.103490e+09,0
36,1821.165991,45.059937,1830.960170,72.960356,83.721844,9.848631,60.193710,-27.039644,75.494978,2.596435e+09,0
37,1818.950989,-22.150024,1819.160902,5.527506,54.445468,3.907562,38.585955,-94.472494,-145.729131,1.374155e+09,1


In [78]:
len(stocks_indicators['AAPL'])

2735

## Calculation of weight table to depict increase and decrease in Stock Points 

In [81]:
def weighs_table(stocks, indices, period):
    stocks_table = pd.DataFrame()
    indices_table = pd.DataFrame()
    for j in stocks:
        weighs_1 = []
        for i in range(1,period+1):
            stocks_indicators = get_stocks_indicators(stocks, i)
            weighs_1.append((len(stocks_indicators[j][stocks_indicators[j]['pct_change']=='1'])/\
                            float(len(stocks_indicators[j])))*100)
        stocks_table = pd.concat([stocks_table, pd.DataFrame(weighs_1)], axis=1)
    for j in indices:
        weighs_1 = []
        for i in range(1,period+1):
            indices_indicators = get_indices_indicators(indices, i)
            weighs_1.append((len(indices_indicators[j][indices_indicators[j]['pct_change']=='1'])/\
                            float(len(indices_indicators[j])))*100)
        indices_table = pd.concat([indices_table, pd.DataFrame(weighs_1)], axis=1)
    stocks_table.index = range(1,period+1)
    stocks_table.columns = stocks.keys()
    indices_table.index = range(1,period+1)
    indices_table.columns = indices.keys()
    return stocks_table, indices_table

In [82]:
stocks_table, indices_table = weighs_table(stocks, indices, 20)

In [83]:
stocks_table.iplot(kind='bar', subplots=True, dimensions=(950,500), title='Percentage of the Increase Stocks Data Points')

In [84]:
indices_table.iplot(kind='bar', subplots=True, dimensions=(950,500), title='Percentage of the Increase Indices Data Points')

# Implementation of Random Forest Classifier Algorithm

In [85]:
def avg_score(x_train, y_train,x_test,y_test,trees):
    accuracy = []
    f1 = []
    rf_model = RandomForestClassifier(trees)
    for i in range(5):
        rf_model.fit(x_train,y_train)
        accuracy.append(rf_model.score(x_test,y_test))
        f1.append(f1_score(y_test,rf_model.predict(x_test), pos_label='1'))
    avg_accuracy = sum(accuracy)/len(accuracy)
    avg_f1 = sum(f1)/len(f1)
    return avg_accuracy, avg_f1

## Using Random Forest Algorithm for prediction of stock and stock price movement 

In [86]:
def stocks_accuracy(stocks, trees, period):
    table_accuracy = pd.DataFrame()
    table_f1 = pd.DataFrame()
    for j in stocks:
        accuracy_values = []
        f1_values = []
        for i in range(1,period+1):
            stocks_indicators = get_stocks_indicators(stocks, i)
            train, test = train_test_split(stocks_indicators[j])
            accuracy, f1 = avg_score(train.iloc[:,:-1],train.iloc[:,-1],test.iloc[:,:-1],test.iloc[:,-1],trees)
            accuracy_values.append(accuracy)
            f1_values.append(f1)
        table_accuracy = pd.concat([table_accuracy, pd.DataFrame({j : accuracy_values})], axis=1)
        table_f1 = pd.concat([table_f1, pd.DataFrame({j : f1_values})], axis=1)
    table_accuracy.index = range(1,period+1)
    table_f1.index = range(1,period+1)
    return table_accuracy, table_f1

def indices_accuracy(indices, trees, period):
    table_accuracy = pd.DataFrame()
    table_f1 = pd.DataFrame()
    for j in indices:
        accuracy_values = []
        f1_values = []
        for i in range(1,period+1):
            indices_indicators = get_indices_indicators(indices, i)
            train, test = train_test_split(indices_indicators[j])
            accuracy, f1 = avg_score(train.iloc[:,:-1],train.iloc[:,-1],test.iloc[:,:-1],test.iloc[:,-1],trees)
            accuracy_values.append(accuracy)
            f1_values.append(f1)
        table_accuracy = pd.concat([table_accuracy, pd.DataFrame({j : accuracy_values})], axis=1)
        table_f1 = pd.concat([table_f1, pd.DataFrame({j : f1_values})], axis=1)
    table_accuracy.index = range(1,period+1)
    table_f1.index = range(1,period+1)
    return table_accuracy, table_f1

In [135]:
stocks_accuracy_table, stocks_f1_table = stocks_accuracy(stocks, 300, 20)

In [116]:
indices_accuracy_table, indices_f1_table = indices_accuracy(indices, 500, 20)

## Graphical Representation of Accuracy and F1 scores of the used algorithm 

In [131]:
stocks_accuracy_table.iplot(dimensions=(950,400), xTitle='Days Ahead', yTitle='Average Score', title='Accuracy scores')

In [117]:
indices_accuracy_table.iplot(dimensions=(950,400), xTitle='Days Ahead', yTitle='Average Score', title='Accuracy scores')

In [132]:
stocks_f1_table.iplot(dimensions=(950,400), xTitle='Days Ahead', yTitle='Average Score', title='F1 scores')

In [118]:
indices_f1_table.iplot(dimensions=(950,400), xTitle='Days Ahead', yTitle='Average Score', title='F1 scores')

In [43]:
def highlight_max(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.max()
    return ['background-color: yellow' if v else '' for v in is_max]

## Tabular Representation of our final calculated results 

In [133]:
stocks_accuracy_table.style.apply(highlight_max, axis=0)

,AAPL,AMZN,GOOG
1,0.486842,0.508772,0.502924
2,0.635673,0.597661,0.59269
3,0.680994,0.655848,0.637135
4,0.69429,0.680527,0.666764
5,0.701318,0.714202,0.699561
6,0.727086,0.715666,0.693704
7,0.720937,0.738507,0.721816
8,0.758651,0.734311,0.732551
9,0.778299,0.770674,0.74956
10,0.754545,0.765982,0.717889


In [119]:
indices_accuracy_table.style.apply(highlight_max, axis=0)

,GSPC,NDX,DJI
1,0.499123,0.531871,0.530409
2,0.602339,0.595322,0.607602
3,0.667544,0.632749,0.641228
4,0.675842,0.65183,0.67877
5,0.678477,0.700439,0.711567
6,0.702782,0.727672,0.677306
7,0.704832,0.73265,0.728551
8,0.724927,0.768035,0.728152
9,0.741935,0.756891,0.748974
10,0.729912,0.763343,0.726393


In [134]:
stocks_f1_table.style.apply(highlight_max, axis=0)

,AAPL,AMZN,GOOG
1,0.530863,0.532034,0.533383
2,0.688198,0.641093,0.648668
3,0.730261,0.699633,0.683456
4,0.739642,0.724693,0.726577
5,0.759433,0.759132,0.747257
6,0.781729,0.761486,0.753886
7,0.779654,0.783505,0.767604
8,0.812936,0.780734,0.778419
9,0.830722,0.812817,0.793517
10,0.804209,0.814072,0.764451


In [121]:
indices_f1_table.style.apply(highlight_max, axis=0)

,GSPC,NDX,DJI
1,0.558164,0.614698,0.58239
2,0.678478,0.676932,0.686307
3,0.738544,0.714545,0.708903
4,0.741054,0.722912,0.731823
5,0.751819,0.775701,0.763498
6,0.766931,0.796481,0.736241
7,0.768812,0.799103,0.783763
8,0.792393,0.821166,0.787527
9,0.808535,0.811124,0.80839
10,0.795015,0.821491,0.794451
